Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.core_expr_types.lambda_maps</a>
========

In [1]:
import proveit
from proveit import Lambda, Conditional
from proveit import x, fx, gx, Qx
from proveit.logic import Equals, Forall
%begin demonstrations

### Lambda substitution

In [2]:
universal_eq = Forall(x, Equals(fx, gx))

In [3]:
Lambda(x, fx).substitution(universal_eq, assumptions=[universal_eq])

{forall_{x} (f(x) = g(x))} |- [x -> f(x)] =  \\ [x -> g(x)]

In [4]:
universal_eq_with_cond = Forall(x, Equals(fx, gx), condition=Qx)

In [5]:
Lambda(x, Conditional(fx, Qx)).substitution(universal_eq_with_cond, 
                                            assumptions=[universal_eq_with_cond])

{forall_{x | Q(x)} (f(x) = g(x))} |- [x -> {f(x) if Q(x).] =  \\ [x -> {g(x) if Q(x).]

In [6]:
%end demonstrations